# Библиотеки

## Компоненты

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!pip install tensorflow==2.3 -q

In [ ]:
#аугментации изображений
!pip install albumentations -q

## Импорт

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import PIL
import cv2
import albumentations

# ML
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostRegressor

# DL
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import EfficientNetB3

# NLP
from gensim.models import doc2vec

# plt
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [ ]:
# фиксируем рандом
RANDOM_SEED = 73
np.random.seed(RANDOM_SEED)

In [ ]:
# from_file = CatBoostRegressor()

# from_file.load_model('../input/car-price-part2-trained-models/model_catboost.cbm')

In [ ]:
# import random
# import os
# import re

In [ ]:
!pip freeze > requirements.txt

## Функции

In [ ]:
# потому что древний sklearn
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
# график loss
def plot_history(history):
    plt.title('Loss')
    plt.plot(history.history['MAPE'], label='train')
    plt.plot(history.history['val_MAPE'], label='test')
    plt.show()

# Данные

## Загружаем предобработанные

In [ ]:
X = pd.read_csv('../input/car-price-part2-trained-models/train_preprocessed.csv').drop(columns='price')
y = pd.read_csv('../input/car-price-part2-trained-models/train_preprocessed.csv', usecols=['price'])

X_test = pd.read_csv('../input/car-price-part2-trained-models/test_preprocessed.csv').drop(columns='price')

submission = pd.read_csv('../input/sf-dst-car-price-prediction-part2/sample_submission.csv')

In [ ]:
X.sample(1).T

## Дополнительная обработка для нейронок

In [ ]:
# эти признаки сделаем one-hot
cat_features_list = ['body_type', 
                     'brand', 
                     'color', 
                     'fuel_type', 
                     'n_doors', 
                     'vehicle_transmission', 
                     'drive_type', 
                     'n_owners', 
                     'full_model_name',
                     'model_year',
                    ]

# эти признаки не трогаем
bool_features_list = ['is_original_techpass', 
                      'is_lefthand_drive',
                     ]

# эти признаки скалируем и нормализуем, если надо
num_features_list = ['engine_displacement', 
                     'engine_power', 
                     'mileage', 
                     'production_year',
                     'ti_own',
                    ]

# эти признаки требуют NLP
text_features_list = ['description']

# этот признак для подгрузки изображения
img_features_list = ['sell_id']

In [ ]:
# объединим для корректной обработки
data = pd.concat([X.assign(source='train'), 
                  X_test.assign(source='test'),
                 ], axis=0, ignore_index=True)
data['description'] = data['description'].fillna('_')  # ннннада
print(X.shape, X_test.shape, data.shape)

In [ ]:
def process_data_v1(df_input):
    df = df_input.copy()
    
    # приводим тип
    for clm in bool_features_list:
        df[clm] = df[clm].astype('uint8')
    
    # нормируем то, что улучшает по результатам исследования
    data['engine_power'] = np.log(data['engine_power'] + 1)
    data['ti_own'] = np.log(data['ti_own'] + 1.1)
#     data['mileage'] = np.log(data['mileage'] + 1)
#     data['production_year'] = np.log(np.max(data['production_year']) - data['production_year'] + 1)
    # скалируем
    scaler = MinMaxScaler()
    for clm in num_features_list:
        df[clm] = scaler.fit_transform(df[[clm]])[:,0]
    # приводим тип
    for clm in num_features_list:
        df[clm] = df[clm].astype('float32')
    
    # категориальные one-hot
    df = pd.get_dummies(df, columns=cat_features_list)
    
    # убираем сложные для простой нейронки
    df.drop(text_features_list + img_features_list, axis = 1, inplace=True)
    
    return df

In [ ]:
# Запускаем и проверяем, что получилось
data_proc = process_data_v1(data)
data_proc.sample(3).T

In [ ]:
data_proc.info()

In [ ]:
# Разделим обратно
X = data_proc.query('source == "train"').drop(columns=['source'])
X_test = data_proc.query('source == "test"').drop(columns=['source'])
print(X.shape, X_test.shape, data_proc.shape)

In [ ]:
X.info()

## Делим на обучение и валидацию

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape)

# v1 полносвязная

Очень простой вариант - просто понять возможности нейронки "в лоб".

## Подготовка

In [ ]:
tf.keras.backend.clear_session()

RANDOM_SEED = 14
np.random.seed(RANDOM_SEED)

filename_now = '../working/best_model.hdf5'
filename_mlp = '../working/best_nn_1_mlp.hdf5'

LR = 1e-2

In [ ]:
mdl_mlp = Sequential([
    L.Dense(1024, input_dim=X_train.shape[1], activation="relu"),
    L.Dropout(0.5),
    L.Dense(256, activation="relu"),
    L.Dropout(0.25),
])

In [ ]:
model = Sequential([
    mdl_mlp,
    L.Dense(1, activation="linear"),
])
mdl_mlp.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(LR, amsgrad=False,)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(filename_now,
                             monitor='val_MAPE', 
                             verbose=1, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_MAPE',
                          patience=20,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=1,
                         )

callbacks_list = [checkpoint, earlystop]

## Обучение

In [ ]:
try:
    model.load_weights(filename_mlp)
except:
    history = model.fit(X_train, y_train,
                        batch_size=128,
                        epochs=1000,    # wait EarlyStopping
                        validation_data=(X_valid, y_valid),
                        callbacks=callbacks_list,
                        verbose=0,
                       )
    plot_history(history)
    model.load_weights(filename_now)
    model.save(filename_mlp)

valid_predict = model.predict(X_valid)
print(f"TEST mape: {(mape(y_valid, valid_predict))*100}%")

## Каггл

In [ ]:
submission['price'] = model.predict(X_test)
submission.to_csv('submission_nn_1.csv', index=False)

# v2 ... + NLP

У меня получился хороший результат через emedding с использованием doc2vec (чуть лучше, чем word2vec с усреднением). Плюс само обучение происходит проще. Результат добавим к полносвязной нейронке из предыдущего варианта.

## Готовим embeddings

In [ ]:
EMBEDDING_SIZE = 300

In [ ]:
# простая токенизация, т.к. текст уже подготовлен
tkn_description = data['description'].str.split()

# корпусы для doc2vec
C_train = [doc2vec.TaggedDocument(tkn_description.iloc[i], [i]) for i in X_train.index]
C_valid = [doc2vec.TaggedDocument(tkn_description.iloc[i], [i]) for i in X_valid.index]
C_test = [tkn_description.iloc[i] for i in X_test.index]

In [ ]:
# сама модель
mdl_d2v = doc2vec.Doc2Vec(vector_size=EMBEDDING_SIZE,
                          min_count=2,  # встречается хотя бы 2 раза
                          seed=RANDOM_SEED,
                         )
# строим словарь
mdl_d2v.build_vocab(C_train)
# и учим
mdl_d2v.train(C_train, 
              total_examples=mdl_d2v.corpus_count, 
              epochs=mdl_d2v.epochs
             )

In [ ]:
# обработаем все тексты
emb_description = tkn_description.map(mdl_d2v.infer_vector)
emb_description.sample()

In [ ]:
V_train = np.array(emb_description.iloc[X_train.index].tolist())
V_valid = np.array(emb_description.iloc[X_valid.index].tolist())
V_test = np.array(emb_description.iloc[X_test.index].tolist())
print(V_train.shape)
print(V_valid.shape)
print(V_test.shape)

## Собираем multi-input NN

In [ ]:
RANDOM_SEED = 55
np.random.seed(RANDOM_SEED)

filename_nlp = '../working/best_nn_2_d2v.hdf5'

LR = 1e-3

In [ ]:
mdl_nlp = Sequential([
    L.Dense(1024, input_dim=V_train.shape[1], activation="relu"),
    L.Dropout(0.25),
    L.Dense(256, activation="relu"),
    L.Dropout(0.25),
])

In [ ]:
combinedInput = L.concatenate([mdl_nlp.output, mdl_mlp.output])

# being our regression head
head = L.Dense(128, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[mdl_nlp.input, mdl_mlp.input], outputs=head)

## Обучаем

In [ ]:
optimizer = tf.keras.optimizers.Adam(LR, amsgrad=False,)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(filename_now,
                             monitor='val_MAPE', 
                             verbose=1, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_MAPE',
                          patience=20,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=1,
                         )

callbacks_list = [checkpoint, earlystop]

In [ ]:
try:
    model.load_weights(filename_nlp)
except:
    history = model.fit([V_train, X_train], y_train,
                         batch_size=128,
                         epochs=1000, # until EarlyStopping
                         validation_data=([V_valid, X_valid], y_valid),
                         callbacks=callbacks_list,
                         verbose=0,
                        )
    plot_history(history)
    model.load_weights(filename_now)
    model.save(filename_nlp)

valid_predict = model.predict([V_valid, X_valid])
print(f"TEST mape: {(mape(y_valid, valid_predict))*100}%")

## Каггл

In [ ]:
submission['price'] = model.predict([V_test, X_test])
submission.to_csv('submission_nn_2.csv', index=False)

# v3 ... + Images

### Data

In [ ]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = X_train.head(9)
random_image_paths = data.loc[random_image.index, 'sell_id'].values
random_image_cat = y.loc[random_image.index, 'price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open('../input/sf-dst-car-price-prediction-part2/img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
# у нас чистые изображения одного размера
IMG_SIZE = (320, 240)

def get_image_array(index):
    images = []
    for index, sell_id in enumerate(data.loc[index, 'sell_id'].values):
        image = cv2.imread('../input/sf-dst-car-price-prediction-part2/img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, IMG_SIZE)  # без этого возникают траблы
        images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) # перевёрнем цветовую схему, чтобы не бесило
    images = np.array(images)
    print('images shape', images.shape, 'dtype', images.dtype)
    return(images)

# все изображения в RAM, можем себе позволить
I_train = get_image_array(X_train.index)
I_valid = get_image_array(X_valid.index)
I_test = get_image_array(X_test.index)

In [ ]:
# проверим правильные цвета
plt.figure(figsize = (12,8))
for i in range(9):
    img = I_train[i]
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
print(f'Images in RAM: {(I_train.nbytes+I_valid.nbytes+I_test.nbytes)/1024**3:0.2f} GB')

### albumentations

In [ ]:
from albumentations import (
    HorizontalFlip, 
    MultiplicativeNoise, GaussNoise, JpegCompression,
    MotionBlur, MedianBlur, Blur,
    ShiftScaleRotate,
    OpticalDistortion, GridDistortion, ElasticTransform,
    CLAHE, IAASharpen, IAAEmboss, RandomBrightnessContrast,
    HueSaturationValue,
    OneOf, Compose,
)

# документация здесь https://albumentations.ai/docs/getting_started/image_augmentation/
# default p=0.5
augmentation = Compose([
    HorizontalFlip(),
    OneOf([
        MultiplicativeNoise(),
        GaussNoise(),
        JpegCompression(),
    ]),
    OneOf([
        MotionBlur(),
        MedianBlur(),
        Blur(),
    ]),
    ShiftScaleRotate(rotate_limit=20),
#     OneOf([
#         OpticalDistortion(),
#         GridDistortion(),
#         ElasticTransform(),
#     ]),
    OneOf([
        CLAHE(),
        IAASharpen(),
        IAAEmboss (),
        RandomBrightnessContrast(),
    ]),
    HueSaturationValue(),
], p=1)

# пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = I_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
# def make_augmentations(images):
#   print('применение аугментаций', end = '')
#   augmented_images = np.empty(images.shape)
#   for i in range(images.shape[0]):
#     if i % 200 == 0:
#       print('.', end = '')
#     augment_dict = augmentation(image = images[i])
#     augmented_image = augment_dict['image']
#     augmented_images[i] = augmented_image
#   print('')
#   return augmented_images

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [ ]:
# NLP part
# tokenize = Tokenizer(num_words=MAX_WORDS)
# tokenize.fit_on_texts(data.description)

# def tokenize_(descriptions):
#   return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

# def tokenize_text(text):
#     return tokenize_([text.numpy().decode('utf-8')])[0]

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tf_process_train_dataset_element(image, table_data, text_emb, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
#     [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text_emb), price

def tf_process_val_dataset_element(image, table_data, text_emb, price):
#     [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text_emb), price

D_train = tf.data.Dataset.from_tensor_slices((
    I_train, 
    X_train, 
    V_train, 
    y_train,
    )).map(tf_process_train_dataset_element)

D_valid = tf.data.Dataset.from_tensor_slices((
    I_valid,
    X_valid,
    V_valid,
    y_valid,
    )).map(tf_process_val_dataset_element)

y_test = np.zeros(len(X_test))

D_test = tf.data.Dataset.from_tensor_slices((
    I_test, 
    X_test,
    V_test,
    y_test,
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
next(iter(D_train));
next(iter(D_valid));
next(iter(D_test));

## Собираем модель

In [ ]:
RANDOM_SEED = 16
np.random.seed(RANDOM_SEED)

filename_cnn = '../working/best_nn_3_cnn.hdf5'

LR = 1e-2  # используем callback для управления уменьшением
BATCH_SIZE = 30

In [ ]:
# удалим файл модели, если хотим обучить заново
# os.remove(filename_cnn)

In [ ]:
# efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
# efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

mdl_cnn = EfficientNetB3(
    input_shape = (IMG_SIZE[1], IMG_SIZE[0], 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'avg',    # avg - max - None
    classifier_activation = 'softmax',    # softmax - None
)

In [ ]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate([mdl_cnn.output, mdl_mlp.output, mdl_nlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[mdl_cnn.input, mdl_mlp.input, mdl_nlp.input], outputs=head)

# model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(LR, amsgrad=False,)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(filename_now,
                             monitor='val_MAPE', 
                             verbose=1, 
                             mode='min',
                             save_best_only = True,
                            )

earlystop = EarlyStopping(monitor='val_MAPE',
                          patience=25,
                          min_delta = 0.001,
                          restore_best_weights=True,
                          verbose=1,
                         )

reduce_lr = ReduceLROnPlateau(
    monitor='val_MAPE',
    factor=0.5,
    patience=5,
    verbose=1,
    mode='min',
    min_delta=0.001,
    cooldown=0,
    min_lr=0,
)

callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
try:
    model.load_weights(filename_cnn)
except:
    history = model.fit(D_train.batch(BATCH_SIZE),
#                          batch_size=128,
                         epochs=1000, # until EarlyStopping
                         validation_data=D_valid.batch(BATCH_SIZE),
                         callbacks=callbacks_list,
                         verbose=0,
                        )
    plot_history(history)
    model.load_weights(filename_now)
    model.save(filename_cnn)

valid_predict = model.predict(D_valid.batch(BATCH_SIZE))
print(f"TEST mape: {(mape(y_valid, valid_predict))*100}%")

## Кагл

In [ ]:
submission['price'] = model.predict(D_test.batch(BATCH_SIZE))
submission.to_csv('submission_nn_3.csv', index=False)

In [ ]:
# history = model.fit(D_train.batch(30),
#                     epochs=100,
#                     validation_data = D_valid.batch(30),
#                     callbacks=callbacks_list
#                    )

In [ ]:
# plt.title('Loss')
# plt.plot(history.history['MAPE'], label='train')
# plt.plot(history.history['val_MAPE'], label='test')
# plt.show();

In [ ]:
# model.load_weights('../working/best_model.hdf5')
# model.save('../working/nn_final.hdf5')

In [ ]:
# test_predict_nn3 = model.predict(D_valid.batch(30))
# print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
# sub_predict_nn3 = model.predict(sub_dataset.batch(30))
# sample_submission['price'] = sub_predict_nn3[:,0]
# sample_submission.to_csv('nn3_submission.csv', index=False)


#### Общие рекомендации:
* Попробовать разные архитектуры
* Провести более детальный анализ результатов
* Попробовать различные подходы в управление LR и оптимизаторы
* Поработать с таргетом
* Использовать Fine-tuning

#### Tabular
* В нейросеть желательно подавать данные с распределением, близким к нормальному, поэтому от некоторых числовых признаков имеет смысл взять логарифм перед нормализацией. Пример:
`modelDateNorm = np.log(2020 - data['modelDate'])`
Статья по теме: https://habr.com/ru/company/ods/blog/325422

* Извлечение числовых значений из текста:
Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.

* Cокращение размерности категориальных признаков
Признак name 'name' содержит данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Можно удалить эти данные. Затем можно еще сильнее сократить размерность, например выделив наличие xDrive в качестве отдельного признака.

* Поработать над Feature engineering



#### NLP
* Выделить из описаний часто встречающиеся блоки текста, заменив их на кодовые слова или удалив
* Сделать предобработку текста, например сделать лемматизацию - алгоритм ставящий все слова в форму по умолчанию (глаголы в инфинитив и т. д.), чтобы токенайзер не преобразовывал разные формы слова в разные числа
Статья по теме: https://habr.com/ru/company/Voximplant/blog/446738/
* Поработать над алгоритмами очистки и аугментации текста



#### CV
* Попробовать различные аугментации
* Fine-tuning

# Blend

In [ ]:
blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

# Model Bonus: проброс признака

In [ ]:
# MLP
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# FEATURE Input
# Iput
productiondate = L.Input(shape=[1], name="productiondate")
# Embeddings layers
emb_productiondate = L.Embedding(len(X.productionDate.unique().tolist())+1, 20)(productiondate)
f_productiondate = L.Flatten()(emb_productiondate)

In [ ]:
combinedInput = L.concatenate([model_mlp.output, f_productiondate,])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_mlp.input, productiondate], outputs=head)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
history = model.fit([X_train, X_train.productionDate.values], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([X_test, X_test.productionDate.values], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model.hdf5')
test_predict_nn_bonus = model.predict([X_test, X_test.productionDate.values])
print(f"TEST mape: {(mape(y_test, test_predict_nn_bonus[:,0]))*100:0.2f}%")

In [ ]:
# 